In [ ]:
# noinspection PyUnresolvedReferences
import torch
# noinspection PyUnresolvedReferences
import torch.nn as nn
# noinspection PyUnresolvedReferences
import torch.optim as optim
# noinspection PyUnresolvedReferences
from torch.utils.data import Dataset, DataLoader
# noinspection PyUnresolvedReferences
import torchvision.transforms.v2 as transforms
# noinspection PyUnresolvedReferences
import torchvision.io as tv_io
# noinspection PyUnresolvedReferences
import glob
# noinspection PyUnresolvedReferences
import json
# noinspection PyUnresolvedReferences
from PIL import Image
# noinspection PyUnresolvedReferences
# noinspection PyUnresolvedReferences
from torchvision.models import efficientnet_b0
# noinspection PyUnresolvedReferences
from torchvision.models import EfficientNet_B0_Weights

import train_utils

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

In [ ]:
pretrained_weights = EfficientNet_B0_Weights.DEFAULT
base_model = efficientnet_b0(weights=pretrained_weights)
base_model.to(device)

In [ ]:
# Transforms, which had been used when training the model
pre_transforms = pretrained_weights.transforms()
# pre_transforms

In [ ]:
num_classes = train_utils.count_folders("../stanford/data/train")
print(f'Number of different car brands: {num_classes}')

# adaptation for our task
base_model.classifier[0] = nn.Dropout(p=0.91, inplace=True)
base_model.classifier[1] = nn.Linear(1280, num_classes)
model = base_model

# model = nn.Sequential(
#     # base_model.features,
#     # base_model.avgpool,
#     # nn.Flatten(),
#     # base_model.classifier[0:3],
#     # nn.Linear(4096, 1024),
#     # nn.ReLU(),
#     # nn.Dropout(.5),
#     # nn.Linear(1024, 512),
#     # nn.ReLU(),
#     base_model,
#     nn.SiLU(inplace=True),
#     nn.Dropout(.2, inplace=True),
#     nn.Linear(1000, 512),
#     nn.SiLU(inplace=True),
#     nn.Dropout(.2, inplace=True),
#     nn.Linear(512, N_CLASSES)
# )

for idx, param in enumerate(model.parameters()):
    # The model will not be fine-tuned, experience showed that it's not beneficial
    assert param.requires_grad

In [ ]:
# All the car brands available in the dataset. Maybe someday the model will be trained on all of them...
# labels = [
#     "Acura", "Alfa_Romeo", "Aston_Martin", "Audi", "Bentley",
#     "BMW", "Bugatti", "Buick", "Caterham", "Chevrolet",
#     "Chrysler", "Citroen", "Dacia", "Dodge", "Ferrari",
#     "Fiat", "Ford", "GMC", "Honda", "Hyundai",
#     "Infiniti", "Isuzu", "Jaguar", "Kia", "Koenigsegg",
#     "Lamborghini", "Land_Rover", "Lexus", "Lotus", "Maserati",
#     "Mazda", "McLaren", "Mercedes-Benz", "Mini", "Mitsubishi",
#     "Morgan", "Nissan", "Opel", "Pagani", "Peugeot",
#     "Porsche", "Renault", "Rolls_Royce", "Saab", "Seat",
#     "Skoda", "Smart", "Subaru", "Suzuki", "Tata",
#     "Tesla", "Toyota", "Volkswagen", "Volvo"
# ]

# But for now, let's stick to those
labels = ["Audi", "BMW", "Chevrolet", "Fiat", "Honda", "Mazda", "Mercedes-Benz", "Toyota",
               "Volvo"]

assert num_classes == len(labels)

In [ ]:
batch_size = 64

train_path = "../stanford/data/train/"
train_dataset = train_utils.DatasetFromFolders(train_path, labels, device, pre_transforms)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
train_N = len(train_loader.dataset)

valid_path = "../stanford/data/valid/"
valid_dataset = train_utils.DatasetFromFolders(valid_path, labels, device, pre_transforms)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
valid_N = len(valid_loader.dataset)

In [ ]:
loss_function = nn.CrossEntropyLoss()

# default value is lr=0.001
optimizer = optim.Adam(model.parameters(), lr=0.0004)
# optimizer = torch.optim.SGD(
#     model.parameters(),
#     lr=0.001,        # Learning rate (start with something small, like 0.001 or 0.01)
#     momentum=0.9,    # Momentum (usually set to 0.9 or 0.8)
#     weight_decay=1e-4  # Optional L2 regularization (weight decay)
# )

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.4, patience=0, min_lr=1e-6)

# DO NOT COMPILE ON OLDER GRAPHIC CARDS!
# my_model = torch.compile(model.to(device))

# Instead, simply send it to the GPU
model = model.to(device)

In [ ]:
IMG_WIDTH, IMG_HEIGHT = (224, 224)

# Small data augmentation to improve model generalization
data_augment = transforms.Compose([
    transforms.RandomRotation(5),
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(.65, 1), ratio=(1, 1)),
    transforms.RandomHorizontalFlip(),
    # transforms.ColorJitter(hue=.02)#brightness=.05, contrast=.05, saturation=.05,hue=.05)
])

In [ ]:
# And now the training!
num_epochs = 15

train_utils.train_and_valid(model, train_loader, valid_loader,
                      data_augment, optimizer, lr_scheduler, loss_function, num_epochs)

In [ ]:
train_utils.model_prediction(model, labels, "../test/test3.jpg", device, pre_transforms)